In [1]:
from app.core.db import Scoped_Session
import dspy
from dotenv import load_dotenv
import os
import openai
import warnings
from sqlalchemy.exc import SAWarning

load_dotenv()

warnings.filterwarnings("ignore", category=SAWarning)

session = Scoped_Session()

turbo = dspy.OpenAI(model='gpt-4o', api_key=os.getenv("OPENAI_API_KEY"), max_tokens=4096)
dspy.settings.configure(lm=turbo)

plan_model = "gpt-3.5-turbo-1106"
generate_model = "gpt-4o-mini"

/home/zhaiyl/miniconda3/envs/tidbai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from app.rag.knowledge_graph.graph_store import TiDBGraphStore
from app.rag.knowledge_graph import KnowledgeGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType

_embed_model = OpenAIEmbedding(
    model=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL
)

graph_store = TiDBGraphStore(
    dspy_lm=turbo,
    session=session,
    embed_model=_embed_model,
)
graph_index =  KnowledgeGraphIndex = KnowledgeGraphIndex.from_existing(
    dspy_lm=turbo,
    kg_store=graph_store,
)

def retrieve_knowledge_graph(query):
    return graph_index.retrieve_with_weight(
        query,
        [],
        depth=1,
    )

In [3]:
from app.rag.vector_store.tidb_vector_store import TiDBVectorStore
from llama_index.core import VectorStoreIndex

vector_store = TiDBVectorStore(session=session)
vector_index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=_embed_model
)

def retrieve_knowledge_embedded_chunks(query, top_k=5):
    retriver = vector_index.as_retriever(
        similarity_top_k=5
    )

    nodes = retriver.retrieve(query)
    return [node.text for node in nodes]

In [4]:
def llm_generate(prompt):
    completion = openai.OpenAI().chat.completions.create(
        model=generate_model,
        messages=[{
            "role": "user",
            "content": prompt
        }],
    )

    return completion.choices[0].message.content

In [5]:
from dotenv import load_dotenv
import openai
from pydantic import BaseModel
from typing import List, Dict, Optional

load_dotenv()

system_instruction = """
You are an intelligent assistant designed to analyze user queries by performing the following tasks:

1. **Analyze the Question**:
    - Break down the main question into a dependency graph that outlines the key components and their relationships.

2. **Break Down into Subquestions**:
    - Decompose the main question into smaller, specific, and manageable subquestions that are conducive to information retrieval.
    - Ensure that each subquestion is concrete and directly related to fetching necessary information.
    - Identify dependencies between subquestions where the answer to one subquestion is required to formulate or answer another.

3. **Generate an Action Plan**:
    - For each subquestion, create a corresponding action step to answer it.
    - Specify the appropriate tool to be used, any necessary arguments, and output tags for each step.
    - Ensure that dependent steps correctly reference the outputs of their prerequisite steps using `{output_tag}` placeholders.

**Available APIs/Tools**:

1. **Knowledge Graph Tool**:
   - **Function**: `retrieve_knowledge_graph(query)`
   - **Description**: Retrieves structured knowledge in the form of a graph, focusing on entities and their relationships.
   - **Use Cases**:
     - Entity Queries
     - Relationship Navigation

2. **Knowledge Embedded Chunks Tool**:
   - **Function**: `retrieve_knowledge_embedded_chunks(query)`
   - **Description**: Retrieves detailed source data based on content similarity, suitable for in-depth or context-heavy queries.
   - **Use Cases**:
     - Content Queries
     - Context Retrieval

3. **LLM Generate Tool**:
   - **Function**: `llm_generate(prompt)`
   - **Description**: Generates text based on the provided prompt and retrieved data using a large language model.
   - **Use Cases**:
     - Summarizing information based on retrieved data
     - Generating answers based on retrieved data
     - Formulating comparative analyses 

**Instructions**:

- **Subquestions Specificity**:
  - Each subquestion should be specific and aimed at retrieving precise information.
  - Avoid vague or overly broad subquestions that may hinder effective information retrieval.

- **Handling Dependencies**:
  - Identify and outline dependencies between subquestions.
  - Ensure that subquestions requiring information from previous steps reference them appropriately using `{output_tag}` placeholders in their `arguments`.

- **Utilizing Tools Effectively**:
  - Select the most appropriate tool for each subquestion based on its nature.
  - Ensure that `arguments` for each tool are correctly populated, incorporating any necessary data from dependent steps.

- **Output Structure**:
  - Use `output_tags` to uniquely identify the output of each step.
  - Reference these tags in subsequent steps to maintain the flow of information and dependencies.

**Example**:

*User Query*: "Could you summary the performance improvement of TiDB in the newest version."

*Generated Action Plan*:

```python
[
    Step(
        id=1,
        subquestion='What is the latest version of TiDB?',
        tool_used='retrieve_knowledge_graph',
        arguments={'query': 'Latest version of TiDB'},
        output_tags='latest_version_graph_data'
    ),
    Step(
        id=2,
        subquestion='What are the performance improvements in the newest TiDB version?',
        tool_used='llm_generate',
        arguments={'prompt': 'what is the version number of the TiDB latest version?'},
        output_tags='latest_version'
    ),
    Step(
        id=3,
        subquestion='What are the performance improvements in the newest TiDB version?',
        tool_used='retrieve_knowledge_embedded_chunks',
        arguments={'query': 'TiDB {latest_version} performance improvements'},
        output_tags='tidb_newest_performance'
    ),
    Step(
        id=4,
        subquestion='Summary the performance improvements of TiDB in the newest version.',
        tool_used='llm_generate',
        arguments={'prompt': 'Summary the performance improvements of TiDB in the newest version based on {tidb_newest_performance}.'},
        output_tags='performance_comparison'
    )
]
```

Dependency Graph:

```python
[
    DependencyEdge(from_step=1, to_step=2),
    DependencyEdge(from_step=2, to_step=3),
    DependencyEdge(from_step=3, to_step=4),
]
```
"""

class Step(BaseModel):
    id: int
    subquestion: str
    tool_used: Optional[str] = None
    arguments: Optional[Dict[str, str]] = None
    output_tags: Optional[str] = None

class DependencyEdge(BaseModel):
    from_step: int
    to_step: int

class QuestionAnalysis(BaseModel):
    steps: List[Step]
    dependency_graph: List[DependencyEdge]

user_query = "Could you summarize the performance improvement of TiDB from version 6.5 to the newest version."
messages = [
    {"role": "system", "content": system_instruction},
    {"role": "user", "content": user_query},
]

client = openai.OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=messages,
    response_format=QuestionAnalysis,
)

message = completion.choices[0].message


In [6]:
message.parsed.steps

[Step(id=1, subquestion='What is the newest version of TiDB?', tool_used='retrieve_knowledge_graph', arguments={'query': 'Newest version of TiDB'}, output_tags='newest_version_graph_data'),
 Step(id=2, subquestion='Determine the newest version number of TiDB from the retrieved data.', tool_used='llm_generate', arguments={'prompt': 'Extract the newest version number of TiDB from {newest_version_graph_data}.'}, output_tags='newest_version_number'),
 Step(id=3, subquestion='What are the performance improvements between TiDB version 6.5 and the newest version?', tool_used='retrieve_knowledge_embedded_chunks', arguments={'query': 'TiDB version 6.5 to {newest_version_number} performance improvements'}, output_tags='performance_improvements_data'),
 Step(id=4, subquestion='Summarize the performance improvements of TiDB from version 6.5 to the newest version.', tool_used='llm_generate', arguments={'prompt': 'Summarize the performance improvements of TiDB from version 6.5 to {newest_version_num

In [7]:
message.parsed.dependency_graph

[DependencyEdge(from_step=1, to_step=2),
 DependencyEdge(from_step=2, to_step=3),
 DependencyEdge(from_step=3, to_step=4)]

In [8]:
import re
import copy


# Step Execution
class PlanExecutor:
    def __init__(self):
        self.state = {
            'variables': {}
        }
    
    def log_error(self, message: str):
        print(f"ERROR: {message}")
    
    def log_info(self, message: str):
        print(f"INFO: {message}")
    
    def log_warning(self, message: str):
        print(f"WARNING: {message}")
    
    def save_milestone(self, milestone: str, output):
        print(f"MILESTONE: {milestone}, output: {output}")

    def log_execution(self, message: str):
        print(f"EXECUION: {message}")
    
    def retrieve_knowledge_embedded_chunks(self, query: str, top_k: int = 5) -> str:
        retriver = vector_index.as_retriever(
            similarity_top_k=5
        )

        nodes = retriver.retrieve(query)
        return [node.text for node in nodes]
    
    def llm_generate(self, prompt: str) -> str:
        return llm_generate(prompt)
    
    def retrieve_knowledge_graph(self, query: str) -> str:
        return graph_index.retrieve_with_weight(
            query,
            [],
            depth=1,
        )
    
    # Step Execution Handler
    def execute_step_handler(self, step: Step) -> bool:
        step_type = step.tool_used
        params = step.arguments or {}
    
        self.log_execution(f"execute step {step}")
        if step_type == "retrieve_knowledge_graph":
            query = params.get('query')
            if not query:
                self.log_error("No query provided for 'retrieve_knowledge_graph' instruction.")
                return False, None
            result = self.retrieve_knowledge_graph(query)
            self.state['variables'][step.output_tags] = result
            self.save_milestone(f"AfterStep{step.id}_KnowledgeGraphRetrieval", result)
            return True, result
    
        elif step_type == "retrieve_knowledge_embedded_chunks":
            query = params.get('query')
            top_k = int(params.get('top_k', 5))
            if not query:
                self.log_error("No query provided for 'retrieve_knowledge_embedded_chunks' instruction.")
                return False, None
            result = self.retrieve_knowledge_embedded_chunks(query, top_k)
            self.state['variables'][step.output_tags] = result
            self.save_milestone(f"AfterStep{step.id}_EmbeddedChunksRetrieval", result)
            return True, result
    
        elif step_type == "llm_generate":
            prompt = params.get('prompt')
            if not prompt:
                self.log_error("No prompt provided for 'llm_generate' instruction.")
                return False, None
            result = self.llm_generate(prompt)
            self.state['variables'][step.output_tags] = result
            self.save_milestone(f"AfterStep{step.id}_LLMGeneration", result)
            return True, result
    
        else:
            self.log_warning(f"Unknown step type: {step_type}")
            return False, None
    
    # Plan Execution
    def execute_plan(self, plan: QuestionAnalysis) -> bool:
        plan = copy.deepcopy(plan)
        self.log_info("Starting plan execution.")
        # Determine execution order based on dependency graph
        # Simple approach: execute steps in the order they appear, assuming dependencies are met
        for step in plan.steps:
            # Replace placeholders in arguments
            if step.arguments:
                for key, value in step.arguments.items():
                    placeholders = re.findall(r'\{(.*?)\}', value)
                    for tag in placeholders:
                        replacement = self.state['variables'].get(tag)
                        if replacement:
                            value = value.replace(f'{{{tag}}}', str(replacement))
                        else:
                            self.log_error(f"Missing value for placeholder '{tag}' in step {step.id}.")
                            return False, None
                    step.arguments[key] = value
            
            # Execute the step
            success, result = self.execute_step_handler(step)
            if not success:
                self.log_error(f"Execution failed at step {step.id}.")
                return False, None
        
        self.state['goal_completed'] = True
        self.log_info("Plan executed successfully.")
        return True, result

In [9]:
executor = PlanExecutor()
success, result = executor.execute_plan(message.parsed)

if success:
    print("\nFinal Summary:")
    print(result)
else:
    print("\nPlan execution failed.")

INFO: Starting plan execution.
EXECUION: execute step id=1 subquestion='What is the newest version of TiDB?' tool_used='retrieve_knowledge_graph' arguments={'query': 'Newest version of TiDB'} output_tags='newest_version_graph_data'


MILESTONE: AfterStep1_KnowledgeGraphRetrieval, output: ([{'id': 63811, 'name': 'v6.3.0', 'description': 'v6.3.0 is a version of TiDB.', 'meta': None, 'entity_type': <EntityType.original: 'original'>}, {'id': 32838, 'name': 'latest stable version', 'description': 'The latest stable version is the most recent release of a component that has been thoroughly tested and is considered production-ready. It is the most recent release that is considered stable and ready for production use. ', 'meta': None, 'entity_type': <EntityType.original: 'original'>}, {'id': 39054, 'name': 'TiDB', 'description': 'TiDB is a distributed SQL database that is compatible with MySQL. It is optimized to make the logging more MySQL-compatible.  Parameters are different from MySQL.', 'meta': None, 'entity_type': <EntityType.original: 'original'>}, {'id': 13263, 'name': 'TiDB 5.4.3', 'description': 'TiDB 5.4.3 is a specific version of the TiDB database, released on October 13, 2022.', 'meta': None, 'entity_type': <E